<h1 style="text-align: center;"> Model Pre-Processing </p>

## Notebook Description

In this notebook, time-series data are merged for SARIMAX modeling of oil stock performance as part of the requirements of the RMDS 2021 Data Science Competition.

<a id='Table-of-Contents'></a>


##  Table of contents
1. [Required Libraries](#Required-Libraries)
2. [Load Data](#Load-Data)
3. [Format Column Names](#Format-Column-Names)
4. [Set Timestamp Range & Frequency](#Set-Timestamp-Range-&-Frequency)
5. [Merge Data](#Merge-DataFrames)
6. [Manage Missing Data](#Missing-Data)
7. [Save Data to CSV](#Save-Data)

## Required Libraries

This notebook uses several Python libraries such as:

In [22]:
# Load required packages 
import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

<a id='Load-Data'></a>

---
## Load Data

[[ go back to the top ]](#Table-of-Contents)

In [23]:
# Load Data Function
def LOAD_DATA(filepath, filename):
    # Read CSV files
    if filename.endswith('.csv'):
        new_df = pd.read_csv(filepath+filename)

    # Read Excel files
    elif filename.endswith('.xlsx'):
        new_df = pd.read_excel(filepath+filename)
    print(type(new_df.index))
    if type(new_df.index) != pd.core.indexes.datetimes.DatetimeIndex:
        for col in new_df.columns:
            if col.lower().find('date') != -1:
                print(f"TIMESTAMP FOUND! '{col}'")
                print()
                new_df['date'] = pd.to_datetime(new_df[col]) # format = '%Y/%m/%d'
                new_df.set_index('date', inplace = True)
                # If datetime col was already == 'date', no need to drop col after set_index, otherwise...
                if col != 'date':
                    new_df.drop(columns = col, inplace = True)
                
    # Try to identify the date column
    elif type(new_df.index) == pd.core.indexes.datetimes.DatetimeIndex:
        print('Index already in datetime')
        
    display(new_df.info())
    return new_df

In [24]:
# Load International Sentiment Data

fpath = '../../data/News_AI_Sentiments/'
fname = 'daily-news-sentiment-international.csv'

sentiment_int = LOAD_DATA(filepath = fpath, filename = fname)
#sentiment_int

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'DateTime'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 395 entries, 2000-04-17 to 2021-02-27
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Daily News Sentiment  395 non-null    float64
dtypes: float64(1)
memory usage: 6.2 KB


None

In [25]:
# Load North American Sentiment Data

fpath = '../../data/News_AI_Sentiments/'
fname = 'daily-news-sentiment-NA.csv'

sentiment_na = LOAD_DATA(filepath = fpath, filename = fname)
#sentiment_na

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'DateTime'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 396 entries, 2013-07-26 to 2021-02-27
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Daily News Sentiment  396 non-null    float64
dtypes: float64(1)
memory usage: 6.2 KB


None

In [26]:
# Load Stock-Closing-Price by Company Data
fpath = '../../'
fname = 'closing_price_by_company.csv'
closing_price = LOAD_DATA(filepath = fpath, filename = fname)
#closing_price

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'Date'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 985 entries, 2017-03-21 to 2021-02-25
Data columns (total 10 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Value_PHILLIPS 66                        985 non-null    float64
 1   Value_BP P.L.C.                          985 non-null    float64
 2   Value_VALERO ENERGY CORPORATION          985 non-null    float64
 3   Value_CHEVRON CORPORATION                985 non-null    float64
 4   Value_OCCIDENTAL PETROLEUM CORPORATION   985 non-null    float64
 5   Value_MARATHON OIL CORPORATION           985 non-null    float64
 6   Value_PIONEER NATURAL RESOURCES COMPANY  985 non-null    float64
 7   Value_CONOCOPHILLIPS                     985 non-null    float64
 8   Value_EXXON MOBIL CORPORATION            985 non-null    float64
 9   Value_MARATHON PETROLEUM CORP

None

In [27]:
# Load Google Mobility Trends grouped by Indicator
fpath = '../../data/Transportation/google/'
fname = 'baseline_pct_change.csv'
google_mobility = LOAD_DATA(filepath = fpath, filename = fname)
#google_mobility

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'date'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 382 entries, 2020-02-15 to 2021-03-02
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   workplaces             382 non-null    float64
 1   retail_and_recreation  382 non-null    float64
 2   grocery_and_pharmacy   382 non-null    float64
 3   residential            382 non-null    float64
 4   transit_stations       382 non-null    float64
 5   parks                  382 non-null    float64
dtypes: float64(6)
memory usage: 20.9 KB


None

In [28]:
# Load Dow Jones Indices grouped by Indicator
fpath = '../../data/Financial_Market/'
fname = 'dow_jones.csv'
dow_jones = LOAD_DATA(filepath = fpath, filename = fname)
#dow_jones

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'date'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000 entries, 2017-03-24 to 2021-02-26
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   dow_jones_transportation_average  1000 non-null   float64
 1   dow_jones_composite_average       1000 non-null   float64
 2   s&p_500                           1000 non-null   float64
 3   dow_jones_industrial_average      1000 non-null   float64
 4   dow_jones_utility_average         1000 non-null   float64
dtypes: float64(5)
memory usage: 46.9 KB


None

<a id='Format-Column-Names'></a>

---
## Format Column Names

[[ go back to the top ]](#Table-of-Contents)

In [29]:
# Function to enforce snake case (no spaces, no caps, no DOTs)
def FORMAT_TITLES(dataframe, start = 0, end = 0):
    df = dataframe
    # Remove extra characters in name
    for index, label in enumerate(df.columns):
        # IF not specified, don't remove chars from end
        if end == 0:
            name = label[start:]
        # Remove 'end'-number of chars from end-of-string
        elif end != 0:
            name = label[start:end]
        # Format column names to be in 'snake case'
        formatted_name = name.strip().lower().replace('.', '').replace(" ", "_")
        df.rename(columns = {label:formatted_name}, inplace = True)
    return df

In [30]:
sentiment_int = FORMAT_TITLES(sentiment_int)
sentiment_int.rename(columns = {'daily_news_sentiment':'int_news_sentiment'}, inplace = True)
#sentiment_int

In [31]:
sentiment_na = FORMAT_TITLES(sentiment_na)
sentiment_na.rename(columns = {'daily_news_sentiment':'na_news_sentiment'}, inplace = True)
#sentiment_na

In [32]:
closing_price = FORMAT_TITLES(closing_price, start = 6)
#closing_price

In [33]:
google_mobility = FORMAT_TITLES(google_mobility)
#google_mobility

In [34]:
dow_jones = FORMAT_TITLES(dow_jones)
#dow_jones

---
## Set Timestamp Range & Frequency

[[ go back to the top ]](#Table-of-Contents)

In [35]:
# Identify all DataFrames in this Notebook
%who DataFrame

closing_price	 df_0	 df_1	 df_i	 df_na	 dow_jones	 google_mobility	 sentiment_int	 sentiment_na	 



In [36]:
df_listed = [closing_price, dow_jones, google_mobility, sentiment_int, sentiment_na]

In [37]:
# Store the latest occurring minimum Timestamp from all the DF's
start = max([df_listed[idx].index[0] for idx in range(len(df_listed))])
start

Timestamp('2020-02-15 00:00:00')

<a id='Merge-Data'></a>

---
## Merge DataFrames

[[ go back to the top ]](#Table-of-Contents)

In [38]:
df_listed = [closing_price, dow_jones, google_mobility, sentiment_int, sentiment_na]
count = 0
for df_i in df_listed:
    # For the first DF, all the others will concat to df_0
    if count == 0:
        # Drop weekend data
        df_0 = df_i[df_i.index.dayofweek < 5]
        # Drop dates before Google Mobililty data begins
        df_0 = df_0.loc[start:].copy()
    else:
        # Drop weekend data
        df_1 = df_i[df_i.index.dayofweek < 5]
        # Concat all the others
        df_0 = pd.concat([df_0, df_1.loc[start:]], axis =1)
    count += 1
display(df_0.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 272 entries, 2020-02-17 to 2021-03-02
Freq: B
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   phillips_66                        244 non-null    float64
 1   bp_plc                             244 non-null    float64
 2   valero_energy_corporation          244 non-null    float64
 3   chevron_corporation                244 non-null    float64
 4   occidental_petroleum_corporation   244 non-null    float64
 5   marathon_oil_corporation           244 non-null    float64
 6   pioneer_natural_resources_company  244 non-null    float64
 7   conocophillips                     244 non-null    float64
 8   exxon_mobil_corporation            244 non-null    float64
 9   marathon_petroleum_corporation     244 non-null    float64
 10  dow_jones_transportation_average   260 non-null    float64
 11  dow_jones_composite_average    

None

<a id='Missing-Data'></a>

---
## Manage Missing Data

[[ go back to the top ]](#Table-of-Contents)

In [39]:
df_na = df_0.fillna({'int_news_sentiment':0})
df_na = df_na.fillna({'na_news_sentiment':0})
df_na = df_na.dropna()
df_na

,phillips_66,bp_plc,valero_energy_corporation,chevron_corporation,occidental_petroleum_corporation,marathon_oil_corporation,pioneer_natural_resources_company,conocophillips,exxon_mobil_corporation,marathon_petroleum_corporation,...,dow_jones_industrial_average,dow_jones_utility_average,workplaces,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,int_news_sentiment,na_news_sentiment
date,,,,,,,,,,,,,,,,,,,,,
2020-02-18,88.00,36.09,82.52,110.24,41.56,10.30,135.56,58.30,59.88,56.47,...,29232.19,960.89,-0.006305,-0.211659,-1.377153,1.345745,2.440042,5.466077,0.00,0.00
2020-02-19,88.36,36.25,83.23,110.74,42.51,10.47,141.17,59.26,60.34,57.69,...,29348.03,950.01,1.001656,2.222899,0.639556,0.345455,2.634615,8.153166,0.00,0.00
2020-02-20,90.19,35.98,84.17,109.81,42.97,10.25,142.53,58.88,59.86,60.26,...,29219.98,952.40,-0.387448,0.192474,-0.167570,1.922258,2.098260,-1.300847,-0.20,0.00
2020-02-21,89.25,35.36,82.90,109.01,42.12,10.11,142.25,58.44,59.13,59.13,...,28992.41,948.74,0.547162,2.786383,-1.632662,1.082046,3.656008,6.725381,0.00,0.00
2020-02-24,86.56,34.06,78.10,104.71,39.48,9.46,135.97,56.38,56.36,55.87,...,27960.80,935.91,3.010224,1.766222,0.129503,0.641213,0.961066,7.336245,0.10,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-19,82.31,22.87,71.53,95.80,25.42,9.45,133.94,48.42,52.37,53.00,...,31494.32,846.51,-26.703297,-20.935180,-12.503418,12.225713,-25.787372,-26.599754,0.00,0.00
2021-02-22,83.96,23.63,74.25,98.39,26.47,10.20,139.47,50.88,54.30,54.85,...,31521.69,830.38,-19.689617,-14.583490,-8.355705,8.119504,-18.301961,-17.634278,0.00,0.00
2021-02-23,85.53,24.23,75.81,99.63,26.06,11.16,145.24,52.10,55.05,55.77,...,31537.35,835.85,-19.324188,-8.923650,-3.151351,6.976641,-14.679208,2.265554,-0.27,-0.50


### Alternate Method

In [40]:
#df_0.fillna({'int_news_sentiment':0}, inplace=True)
#df_0.fillna({'na_news_sentiment':0}, inplace=True)
#df_0.dropna(inplace=True)
#df_0

<a id='Save-Data'></a>

---
## Save Data to CSV

[[ go back to the top ]](#Table-of-Contents)

In [41]:
#df_na.to_csv(r'./model_ready_data.csv', index = True)